<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With `NumPy, pandas & dask`**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## `dask`

From https://dask.org:

> Dask natively scales Python &mdash; Dask provides advanced parallelism for analytics, enabling performance at scale for the tools you love<br><br>
> Dask is open source and freely available. It is developed in coordination with other community projects like NumPy, pandas, and scikit-learn.

## `dask` + `NumPy`

### The Client

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
from dask.distributed import Client

In [ ]:
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='4GB')

In [ ]:
client

### `dask` Array

In [ ]:
import dask.array as da

In [ ]:
x = da.random.standard_normal((15000, 15000),  # total size
                     chunks=(1500, 1500))  # size of NumPy arrays

In [ ]:
x

In [ ]:
x[:2, :5]

In [ ]:
x[:2, :5].compute()  # gives result as ndarray object

In [ ]:
y = x + x.T

In [ ]:
z = y[::2, 5000:].mean(axis=1)

In [ ]:
z

In [ ]:
%time a = z.compute()  # gives the result ...

In [ ]:
a  # ... as NumPy ndarray object

In [ ]:
y = y.persist()  # persisting the data in memory (if enough is available)

In [ ]:
y[0, 0]

In [ ]:
%time y[0, 0].compute()

In [ ]:
%time y.sum()

In [ ]:
%time y.sum().compute()

In [ ]:
client.shutdown()

## `dask` + `pandas` 

### The Client

In [ ]:
from dask.distributed import Client, progress

In [ ]:
client = Client(processes=False, threads_per_worker=4,
                n_workers=2, memory_limit='4GB')

In [ ]:
client

### `dask DataFrame`

In [ ]:
import pandas as pd
import dask.dataframe as dd

In [ ]:
path = '../../../data/'  # adjust the path to you own path
fnc = path + 'data.csv'
fnp = path + 'data.pq'

In [ ]:
ls -n $path

In [ ]:
%time df = pd.read_parquet(fnp)

In [ ]:
df.info()

In [ ]:
ddf = dd.from_pandas(df, chunksize=1000000)

In [ ]:
ddf.info()

In [ ]:
ddf.index

In [ ]:
%time ddf.index.compute()

In [ ]:
ddf = ddf.repartition(freq='1M')

In [ ]:
ddf

In [ ]:
%time df.head()

In [ ]:
%time ddf.head()

In [ ]:
%time df[df['A'] > 0]

In [ ]:
%time ddf[ddf['A'] > 0]

In [ ]:
%time ddf[ddf['A'] > 0].compute()

In [ ]:
%time df[(df['A'] > 0) & (df['B'] < 0)]

In [ ]:
%time ddf[(ddf['A'] > 0) & (ddf['B'] < 0)]

In [ ]:
%time ddf[(ddf['A'] > 0) & (ddf['B'] < 0)].compute()

In [ ]:
%time df.query('A > 0 & B < 0')

In [ ]:
%time ddf.query('A > 0 & B < 0')

In [ ]:
%time ddf.query('A > 0 & B < 0').compute()

In [ ]:
%time df.groupby('M')['C'].mean()

In [ ]:
%time ddf.groupby('M')['C'].mean()

In [ ]:
%time ddf.groupby('M')['C'].mean().compute()

In [ ]:
%time df[['D', 'E']].resample('1w').mean().head()

In [ ]:
%time ddf[['D', 'E']].resample('1w').mean().head()

### Reading Data 

In [ ]:
%time ddf = dd.read_parquet(fnp, chunksize=1000000)

In [ ]:
ddf.info()

In [ ]:
ddf.columns

In [ ]:
ddf.dtypes

In [ ]:
%time df = pd.read_csv(fnc)

In [ ]:
%time df[['B', 'D']].mean()

In [ ]:
%time ddf = dd.read_csv(fnc)

In [ ]:
ddf.columns

In [ ]:
ddf.dtypes

In [ ]:
ddf.head()

In [ ]:
%time ddf[['B', 'D']].mean()

In [ ]:
%time ddf[['B', 'D']].mean().compute()

In [ ]:
%time ddf = ddf.set_index('Unnamed: 0')

In [ ]:
ddf.index = ddf.index.rename('Date')

In [ ]:
ddf.info()

In [ ]:
ddf.head()

In [ ]:
ddf.index.compute()

In [ ]:
client.shutdown()

In [ ]:
ls -n $path

In [ ]:
!rm $path/data.*

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>